In [28]:
async def f():
    print('hi')

In [29]:
await f()

hi


In [30]:
from redis import asyncio as aioredis

In [32]:
r = aioredis.Redis(host="localhost", port=6379)
print(r)

Redis<ConnectionPool<Connection<host=localhost,port=6379,db=0>>>


In [33]:
s = r.pubsub()
print(s)

In [49]:
p = s.subscribe("channel:23")
print(p)

<coroutine object PubSub.subscribe at 0x1122991c0>


In [50]:
import asyncio

In [51]:
c = await p
print(c)

None


In [52]:
await c.wait_message()
j = await c.get(encoding='utf-8')
print(j)

AttributeError: 'NoneType' object has no attribute 'wait_message'

In [2]:
import asyncio
import async_timeout
import redis.asyncio as redis

STOPWORD = "STOP"


async def reader(channel: redis.client.PubSub):
    while True:
        try:
            async with async_timeout.timeout(1):
                message = await channel.get_message(ignore_subscribe_messages=True)
                if message is not None:
                    print(f"(Reader) Message Received: {message}")
                    if message["data"].decode() == STOPWORD:
                        print("(Reader) STOP")
                        break
                await asyncio.sleep(0.01)
        except asyncio.TimeoutError:
            pass


r = redis.from_url("redis://localhost")
pubsub = r.pubsub()
await pubsub.subscribe("channel:1", "channel:2")

future = asyncio.create_task(reader(pubsub))

await r.publish("channel:1", "Hello")
await r.publish("channel:2", "world")
# await r.publish("channel:1", STOPWORD)

await future

(Reader) Message Received: {'type': 'message', 'pattern': None, 'channel': b'channel:1', 'data': b'Hello'}
(Reader) Message Received: {'type': 'message', 'pattern': None, 'channel': b'channel:2', 'data': b'world'}


CancelledError: 

In [ ]:
r.publish("channel:1", "wow")

In [ ]:
await r.publish('channel:1', 'wow')